In [7]:
import pandas as pd 
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
from math import sqrt

plt.style.use('ggplot')

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_process import ArmaProcess

from dataloader import load_data, create_count_data

In [9]:
X_all_30_train = np.load('DeepLearningFinalProject-main/X_all_30_train.npy')
X_all_30_test = np.load('DeepLearningFinalProject-main/X_all_30_test_new.npy')
X_all_7_train = np.load('DeepLearningFinalProject-main/X_all_7_train.npy')
X_all_7_test = np.load('DeepLearningFinalProject-main/X_all_7_test_new.npy')
y_all_30_train = np.load('DeepLearningFinalProject-main/y_all_30_train.npy')
y_all_30_test = np.load('DeepLearningFinalProject-main/y_all_30_test_new.npy')
y_all_7_train = np.load('DeepLearningFinalProject-main/y_all_7_train.npy')
y_all_7_test = np.load('DeepLearningFinalProject-main/y_all_7_test_new.npy')
